# Proyecto Final

# CAMISIM
CAMISIM es un programa para simular datos metagenommicosa partir de genomas, a grandes rasgos este programa toma uno o varios archivos de genomas,y por un proceso decorte aleatorio de secuencias crea un archivo de datos metagenomicos.

A continuación muestro los pasos para hacer una recreaccion de este algoritmos, y exploicare cada una delas funciones creadas.

In [112]:
import math
import random
import numpy as np

from random import seed
from random import randint

La primera funcion se encarga unicamente de leer cada archivo (genoma) y convertirlo en un diccionario en donde las 'llaves' son los nombres de cada genoma y los 'valores' la secuencia de nucleotidos.

In [9]:
# funcion para lectura de archivos
# ingresa un archivo multifasta y arroja una lista con cada secuencia (contig)
dic = {}
def lectura_genoma(File):
    lista = []
    with open(File,'r') as f:
        lines=f.read() # lectura decada linea del archivo
        lines=lines.split('>') # identificador de '>'
        lines=['>'+ x for x in lines[1:]] # lista con cada elemento que comienza con '>'
        for x in lines:
            file_name=x.split('\n')[0][1:] # quita los saltos de linea reemplazandolos con '\n'
            fil=open(file_name+'.fa','w')
            fil.write(x)
            fil.close()
            x2 = x.replace("\n",",",1) # el primer '\n' se reemplazapor una coma
            x3 = x2.replace("\n","") # los siguientes se quitan
            lista.append(x3) # lista con un contig en cada elemento y su identificador
        # convertir la lista en un diccionario
        for x in lista:
            x = x.split(',')
            dic[x[0]] = x[1] 
        return(dic)    

In [10]:
lectura_genoma('/home/csilva/GIT/Tesis_Maestria/Data/new-c-genomes/Clavibacter_michiganensis_subsp_capsici_1101.fna-cortado.fna')

{'>NZ_CP048049.1': '...',
 '>NZ_CP048050.1': '...'}

Esta segunda funcion se encarga de cortar una secuenciade nucleotidos dada, a partir de una pocision y una longitud dada.

In [11]:
# funcion para cortar reads, dado una pocision de inicio y una longitud de corte
def cutout(read,i,n_length): 
    cropped_read = read[i:i+n_length]
    return(cropped_read) 

In [12]:
cutout(dic['>NZ_CP048049.1'],5, 10)

'CGACCGCTCC'

Ahora, la ultima funcion se compone delafuncion anterior para cortar reads, y se complementa, haciendo esto para cada genoma dentro de diccionario creado.

In [154]:
# funcion para cortar los reads aleatoriamente
# Al ingresar un conjunto de genomas, este nos arroja un metagenoma con varios reads tomados de los genomas iniciales
# para esto se le entrega el numero de reads deseados por cada genoma y la longitud.
def reads(diccionario,longitud,num_reads):  # num_reads
    dic = {}
    contig = 0
    k = 0
    while k < num_reads:
        for key in diccionario:#range(len(diccionario.keys())):
            newkey = key + '_' + str(k) 
            contig = random.choice(list(diccionario.values())) # escoje un contig al azar 
            i = randint(1,(len(contig)-longitud)) # toma una pocision de inicio al azar
            dic[newkey] = cutout(contig,i,longitud) # para el contig dado anteriormente, se corta desde la pocision i de lonjitud ln
            # se debe calcular cuantas veces va a escoger cada contig,dependiendo del tamaño del contig inicial
            #print(dic)
        
        k += 1
    file = open("prueba.fastq",'wt')
    for key in dic: 
        file.write(str(key))
        file.write(str('\n'))
        file.write(str(dic[key]))
        file.write(str('\n'))
        file.write(str('+'))
        file.write(str('\n'))
        file.write(str('AAAAA')) #calidad
        file.write(str('\n'))     
    file.close()
    print(dic)

In [155]:
reads(dic,10,3)

{'>NZ_CP048049.1_0': 'GCCTCGAGCA', '>NZ_CP048050.1_0': 'CGAGCGATGG', '>NZ_CP048049.1_1': 'AGAATTGCTT', '>NZ_CP048050.1_1': 'GGCGGCTGCA', '>NZ_CP048049.1_2': 'TACAGCATCG', '>NZ_CP048050.1_2': 'CAGATCCAGG'}


A continuación se quiere agrupar todo dentro de una clase, para poder llamar a la clase completa cuando sea necesario, y poder realizar el proceso para varios archivos al mismo tiempo. Empezando con un solo archivo como prueba de la clase.

In [ ]:
class CAMISIM_CAMI1:
    def __init__(self):
        self.dic = {}
        self.list = []
        
    # funcion para lectura de archivos
    # ingresa un archivo multifasta y arroja una lista con cada secuencia (contig)
    
    # 
    def lectura_genoma(File):
        lista = []
        dic = {}
        with open(File,'r') as f:
            lines=f.read() # lectura decada linea del archivo
            lines=lines.split('>') # identificador de '>'
            lines=['>'+ x for x in lines[1:]] # lista con cada elemento que comienza con '>'
            for x in lines:
                file_name=x.split('\n')[0][1:] # quita los saltos de linea reemplazandolos con '\n'
                fil=open(file_name+'.fa','w')
                fil.write(x)
                fil.close()
                x2 = x.replace("\n",",",1) # el primer '\n' se reemplazapor una coma
                x3 = x2.replace("\n","") # los siguientes se quitan
                lista.append(x3) # lista con un contig en cada elemento y su identificador
            # convertir la lista en un diccionario
            for x in lista:
                x = x.split(',')
                dic[x[0]] = x[1] 
            
            print(f"diccionario de genomas {dic}")
    
        
    # funcion para cortar reads, dado una pocision de inicio y una longitud de corte
    def cutout(self,read,i,n_length): 
        cropped_read = read[i:i+n_length]
        return(cropped_read) 
    
    
    #def reads(diccionario,longitud,num_cortes):
    def reads(diccionario,longitud,num_reads):
        dic = {}
        contig = 0
        k = 0
        while k < num_reads:
            for key in diccionario:#range(len(diccionario.keys())):
                contig = random.choice(list(diccionario.values())) # escoje un contig al azar 
                i = randint(1,(len(value)-longitud)) # toma una pocision de inicio al azar
                dic[key] = [cutout(contig,i,longitud)]  # para el contig dado anteriormente, se corta desde la pocision i de lonjitud ln
            k += 1
        print(dic)

In [ ]:
CAMISIM_CAMI1.reads(dic,5,10) 

Y por ultimo realizandole unas pocas modificaciones a la clase, para poder realizar el proceso para varios archivos de genomas.

In [16]:
import math
import random
from random import seed
from random import randint

class CAMISIM_CAMI2:
    def __init__(self):
        self.dic = {}
        self.list = []
        
    # funcion para lectura de archivos
    # ingresa un archivo multifasta y arroja una lista con cada secuencia (contig)
    def lectura_genoma(File):
        lista = []
        dic = {}
        with open(ruta + '/' + File,'r') as f:
            lines=f.read() # lectura decada linea del archivo
            lines=lines.split('>') # identificador de '>'
            lines=['>'+ x for x in lines[1:]] # lista con cada elemento que comienza con '>'
            for x in lines:
                file_name=x.split('\n')[0][1:] # quita los saltos de linea reemplazandolos con '\n'
                fil=open(file_name+'.fa','w')
                fil.write(x)
                fil.close()
                x2 = x.replace("\n",",",1) # el primer '\n' se reemplazapor una coma
                x3 = x2.replace("\n","") # los siguientes se quitan
                lista.append(x3) # lista con un contig en cada elemento y su identificador
            # convertir la lista en un diccionario
            for x in lista:
                x = x.split(',')
                dic[x[0]] = x[1] 
            
            print(f"diccionario de genomas {dic}")
    
        
    # funcion para cortar reads, dado una pocision de inicio y una longitud de corte
    def cutout(self,read,i,n_length): 
        cropped_read = read[i:i+n_length]
        return(cropped_read) 
    
    
    #def reads(diccionario,longitud,num_cortes):
    def reads(diccionario,longitud,num_reads):
        dic = {}
        contig = 0
        k = 0
        while k < num_reads:
            for key in diccionario:#range(len(diccionario.keys())):
                contig = random.choice(list(diccionario.values())) # escoje un contig al azar 
                i = randint(1,(len(value)-longitud)) # toma una pocision de inicio al azar
                dic[key] = [cutout(contig,i,longitud)]  # para el contig dado anteriormente, se corta desde la pocision i de lonjitud ln
            k += 1
        print(dic)

y aplicando lo visto en clase sobre lectura de arhivos, e iteracion sobre cada uno de ellos.

In [ ]:
# 

In [167]:
import os
from pathlib import Path
ruta = '/home/csilva/GIT/Tesis_Maestria/Data/all-c-genomes'
contenido = os.listdir(ruta)
genomas = []
for i in contenido:
    if os.path.isfile(os.path.join(ruta,i)) and i.endswith('.fna'):
        genomas.append(i)
n = len(genomas) # número de genomas (número de archivos)
diccionario_reads_metagenoma = {}



for i in range(n): 
    lectura_genoma(ruta + '/' + genomas[i]) 
    
    
    

In [168]:
genomas

['Clavibacter_michiganensis_subsp_capsici_1101.fna',
 'Clavibacter_michiganensis_subsp_nebraskensis_DOAB_397.fna',
 'Clavibacter_michiganensis_subsp_nebraskensis_7580.fna',
 'Clavibacter_michiganensis_subsp_insidiosus_LMG_3663.fna',
 'Clavibacter_michiganensis_subsp_insidiosus_CFBP_1195.fna',
 'Clavibacter_michiganensis_subsp_nebraskensis_SL1.fna',
 'Clavibacter_michiganensis_subsp_insidiosus_CFBP_6488.fna',
 'Clavibacter_michiganensis_subsp_capsici_1106.fna',
 'Clavibacter_michiganensis_subsp_capsici_PF008.fna',
 'Clavibacter_michiganensis_subsp_insidiosus_R1-1.fna',
 'Clavibacter_michiganensis_subsp_nebraskensis_A6096.fna',
 'Clavibacter_michiganensis_subsp_nebraskensis_44.fna',
 'Clavibacter_michiganensis_subsp_nebraskensis_HF4.fna',
 'Clavibacter_michiganensis_subsp_sepedonicus_ATCC33113.fna',
 'Clavibacter_michiganensis_subsp_nebraskensis_CIBA.fna',
 'Clavibacter_michiganensis_subsp_tessellarius_ATCC_33566.fna',
 'Clavibacter_michiganensis_subsp_insidiosus_CFBP_2404.fna',
 'Clavib